# Лабораторная работа № 2

# Блочное симметричное шифрование

## Вариант

3(в)

## Задание 

Реализовать систему симметричного блочного шифрования,
позволяющую шифровать и дешифровать файл на диске с использованием
блочного шифра RC6 в режиме шифрования PCBC. 

# Константы

$ w $ - размер слова в битах. Блоки открытого текста и шифротекста имеют длину $ 4w $ - бит (т.к. 4 регистра).

$ r $ - количество раундов. Так же размер расширенного массива ключей $ S $ будет иметь $ t=(2r+4) $ слово. Допустимые значения $ 0\leq r\leq 255 $. 

$ b $ - количество байт в секретном ключе $ K $.

$ K $ - секретный ключ размеров $ b $ байт: $K[0], K[1], ... , K[b-1] $.

In [14]:
import math

# Входные данные
w = 32
r = 2
b = 8
K = [0b01010101, 0b10101010, 0b01010101, 0b10101010, 0b11111111, 0b00000000, 0b11110000, 0b00001111]


# Неизменяемые параметры
# количество байт в слове
u = w/8
# Размер расширинного размера ключей
t = 2 * r + 4

In [51]:
# Реализация циклических сдвигов вправо/влево

# rotate right input x, by n bits
def ROR(x, n, bits = 32):
    mask = (2**n) - 1
    mask_bits = x & mask
    return (x >> n) | (mask_bits << (bits - n))

#rotate left input x, by n bits
def ROL(x, n, bits = 32):
    return ROR(x, bits - n,bits)

# Подготовка ключа

генерация констант

Для конкретного w мы определяем две величины:

$$ P_w \leftarrow Odd((e-2)2^w)\\Q_w \leftarrow Odd((\phi-1)2^w) $$

где $ e=2.71828... $(экспонента), $ \phi=1.61803... $ (золотое сечение). 

$ Odd(\cdot) $ - операция округления до ближайшего нечетного целого.

In [37]:
# операция округления до ближайшего нечетного целого
def odd(x):
    if x - math.trunc(x) == 0:
        return x + 1
    return math.ceil(x) if math.ceil(x) % 2 == 1 else math.floor(x)

# константа золотого сечения
phi = (1 + 5 ** 0.5) / 2

# генерация констант
p_w = odd( (math.e - 2) * math.pow(2, w) )
q_w = odd( (phi - 1) * math.pow(2, w) ) 

print(f'P_w = {bin(p_w)}\nQ_w = {bin(q_w)}')

P_w = 0b10110111111000010101000101100011
Q_w = 0b10011110001101110111100110111001


# Конвертация секретного ключа

На первом этапе нужно скопировать секретный ключ из массива $ K[0...b-1] $ в массив $ L[0...c-1] $, который состоит из $c=b/u$ слов, где $u=w/8$-количество байт в слове. Если   $b$ не кратен $w / 8$, то $L$дополняется нулевыми битами до ближайшего большего кратного:

In [33]:
c = math.ceil(max(b, 1) / u)
print(f'c = {c}')

L = []
for i in range(0, c):
    L.append(0)
    for j in range(0, math.trunc(b/c)):
        L[i] = ROL(L[i], 8) + K[math.trunc(i * u + j)] if i * u + j < b else 0
    print(f'L[{i}] = {bin(L[i])}')
    if i == c - 1:
        break
print(f'Конвертация секретного ключа: {L}')

c = 2
L[0] = 0b1010101101010100101010110101010
L[1] = 0b11111111000000001111000000001111
Конвертация секретного ключа: [1437226410, 4278251535]


# Инициализация массива ключей

Массив ключей $S$ так же называют расширенной таблицей ключей. Она заполняется с помощью тех самых магических констант, которые мы определили ранее:

In [57]:
S = []
S.append(p_w)

for i in range(1, t):
    S.append(S[i-1] + q_w)
    print(f'S[{i}] = {bin(S[i])}')

S[1] = 0b101010110000110001100101100011100
S[2] = 0b111110100010100000100010011010101
S[3] = 0b1010010010100001111011111010001110
S[4] = 0b1100110000101111110011100001000111
S[5] = 0b1111001110111101101011001000000000
S[6] = 0b10001101101001011100010101110111001
S[7] = 0b10100001011011001011010010101110010


# Перемешивание

Этот шаг состоит в том, чтобы перемешать секретный ключ, который нам дал пользователь. 

На выходу получаем w-битный массив ключей S[0,...,t]

In [62]:
A = 0
B = 0
i = 0
j = 0

v = 3 * max(c, t)

for s in range(1, v):
    A = ROL(S[i] + A + B, 3)
    S[i] = A
    B = ROL(L[j] + A + B, (A + B) % 32)
    L[j] = B
    i = (i + 1) % t
    j = (j + 1) % c
    
print(f'Массив ключей S: {S}')

Массив ключей S: [3404431282, 1592753802, 1091041212, 1899076460, 2891079180, 285541210, 958610939, 1707317516]
